In [1]:
import torch
import pickle
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy
from tqdm import tqdm


from model import LR
from data import FairnessDataset, SyntheticDataset, GermanDataset, IncomeDataset
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import *
from ei_model_test import EIModel, fair_batch_proxy, covariance_proxy

# warnings.filterwarnings('ignore')

In [2]:
# dataset = SyntheticDataset(seed=1)
# dataset = GermanDataset(seed=1)
dataset = IncomeDataset(seed=0)

In [5]:
torch.manual_seed(0)
train_tensors, val_tensors, test_tensors = dataset.tensor(z_blind=False)
train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)

model = LR(train_dataset.X.shape[1])

tau = 0.5
delta = 0.5
lamb = 1.
alpha = 0.
effort = Optimal_Effort(delta)
proxy = covariance_proxy
lr = 1e-2
n_epochs = 100

# EI Model
ei_model = EIModel(deepcopy(model), proxy, effort, tau)
ei_model.train(train_dataset, lamb, alpha, lr, n_epochs)
for module in ei_model.model.layers:
    if hasattr(module, 'weight'):
        weights_0 = module.weight.data
    if hasattr(module, 'bias'):
        bias_0 = module.bias.data
theta_0 = torch.cat((weights_0[0], bias_0), 0)
print(theta_0)

# # REI Model
# rei_model = EIModel(deepcopy(model), proxy, effort, tau)
# rei_model.train(train_dataset, lamb, alpha, n_epochs=200, abstol=1e-7)
# for module in rei_model.model.layers:
#     if hasattr(module, 'weight'):
#         weights_r_0 = module.weight.data
#     if hasattr(module, 'bias'):
#         bias_r_0 = module.bias.data
# theta_0_r = torch.cat((weights_r_0[0], bias_r_0), 0)

# print(theta_0_r)

Training [alpha=0.00; lambda=0.00; delta=0.50]: 100%|██████████| 100/100 [03:38<00:00,  2.19s/epochs]

tensor([ 4.7478e-01,  1.5598e-01,  7.7882e-01, -3.4081e-01, -2.6925e-01,
         3.3248e-04, -1.9050e-02,  3.6647e-03, -1.0576e+00, -5.0976e-01,
        -1.0065e+00, -4.3150e-01, -5.9548e-01, -4.9879e-01, -8.0174e-01,
        -8.1878e-01,  1.0859e+00,  7.9898e-01,  3.5037e-01,  8.7859e-01,
         1.0070e+00,  4.7153e-01,  2.9697e-01,  5.0400e-01,  1.2338e+00,
         1.1365e+00,  1.3547e-01,  9.8474e-01,  4.4192e-01,  6.1603e-01,
         5.8151e-01,  4.6593e-02,  5.2171e-02,  8.4468e-01,  3.6631e-01,
         1.3328e+00, -1.8848e-01,  5.2787e-01,  1.1348e-01,  6.5862e-01,
         1.2283e-01,  2.2122e-01,  5.3878e-01,  4.9754e-02,  8.0645e-01,
        -1.8561e-02, -6.8724e-02, -3.5685e-01,  5.7145e-01,  3.1061e-01,
         7.0224e-01,  4.6375e-01,  8.3738e-01,  2.9364e-01,  3.9992e-01,
        -4.1479e-02,  1.3246e+00,  1.0971e+00,  7.6897e-02,  4.5306e-01,
         8.2388e-01,  3.9011e-01,  5.8332e-01,  1.7734e-01,  3.3326e-01,
         2.3058e-01,  1.0258e+00,  9.4341e-01,  4.3

In [6]:
lf_n = 48
alpha = alpha
print(f'Train Parameters')
print('-'*lf_n)
print(f'Dataset                   |   {dataset.__class__.__name__}')
print(f'Proxy                     |   {proxy.__name__}')
print(f'Effort                    |   {effort.__class__.__name__}')
print(f'alpha                     |   {alpha}')
print(f'lambda                    |   {lamb}')
print(f'delta                     |   {delta}')
print(f'GD Epochs                 |   {n_epochs}')
print(f'GD lr                     |   {lr}')
print(f'PGA Epochs                |   until convergence')
print('-'*lf_n)
print('-'*lf_n)
print('Evaluation Results')
print('-'*38)
for alpha_eval in [0., 0.1, 0.5, 1.5, 5.0]:
    Y_hat, Y_hat_max, fair_loss = ei_model.predict(test_dataset, alpha_eval, 1e-7)
    accuracy, ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
    for module in ei_model.model_adv.layers:
        if hasattr(module, 'weight'):
            weights_adv = module.weight.data
        if hasattr(module, 'bias'):
            bias_adv = module.bias.data
    theta_adv = torch.cat((weights_adv[0], bias_adv), 0)
    alphas = (theta_adv-theta_0).abs()

    # Y_hat_r, Y_hat_max_r, fair_loss_r = rei_model.predict(test_dataset, alpha, 1e-7)
    # accuracy_r, rei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat_r, Y_hat_max_r, tau)
    # for module in ei_model.model_adv.layers:
    #     if hasattr(module, 'weight'):
    #         weights_adv_r = module.weight.data
    #     if hasattr(module, 'bias'):
    #         bias_adv_r = module.bias.data
    # theta_adv_r = torch.cat((weights_adv_r[0], bias_adv_r), 0)
    # alphas_r = (theta_adv_r-theta_0_r).abs()

    print(f'alpha                     |   {alpha_eval}')
    print(f'Accuracy                  |   {accuracy:.5f}')
    # print(f'Accuracy (Robust)         |   {accuracy_r:.5f}')
    print(f'Fairness Loss             |   {fair_loss:.5f}')
    # print(f'Fairness Loss (Robust)    |   {fair_loss_r:.5f}')
    print(f'EI Disparity              |   {ei_disparity:.5f}')
    # print(f'EI Disparity (Robust)     |   {rei_disparity:.5f}')
    # print(f'alphas                    |   {np.round(alphas, 4)}')
    # print(f'alphas (Robust)           |   {np.round(alphas_r, 4)}')
    # print(theta_0)
    # print(theta_adv)
    # print(theta_adv_r)
    print('-'*38)

Train Parameters
------------------------------------------------
Dataset                   |   IncomeDataset
Proxy                     |   covariance_proxy
Effort                    |   Optimal_Effort
alpha                     |   0.0
lambda                    |   0.0
delta                     |   0.5
GD Epochs                 |   100
GD lr                     |   0.01
PGA Epochs                |   until convergence
------------------------------------------------
------------------------------------------------
Evaluation Results
--------------------------------------
alpha                     |   0.0
Accuracy                  |   0.63287
Fairness Loss             |   0.00010
EI Disparity              |   0.01070
--------------------------------------
alpha                     |   0.1
Accuracy                  |   0.63287
Fairness Loss             |   0.00085
EI Disparity              |   0.07522
--------------------------------------
alpha                     |   0.5
Accuracy       